# Get the necessary packages

In [ ]:
!git clone https://github.com/your-username/your-repo.git
%cd your-repo

In [ ]:
!pip install -r requirements.txt

# Download the dataset

In [ ]:
import kagglehub

path = kagglehub.dataset_download('abdellah213/sitrex-dataset')

# Imports and Setup

In [ ]:
# Import necessary libraries
import os
from sklearn.model_selection import train_test_split
from sitrex.preprocessing import load_dataset, preprocess_data, SimilarityDataset, UsefulnessDataset
from sitrex.model import angle_usefulness_model, angle_similarity_model, TQDMProgressBar
import tensorflow as tf

import numpy as np
# Define constants
DATASET_PATH = os.path.join(path, 'dataset')
# EarlyStop save path for the Siamese Transformer
MODEL_SAVE_PATH_SIMILARITY = './angle_similarity_model.keras'
# EarlyStop save path for the Angle Usefulness Transformer
MODEL_SAVE_PATH_USEFULNESS = './angle_usefulness_model.keras'
# Maximum sequence length (L)
MAXLEN = 100
# Batch Size
batch_size = 32

# Loading the data

In [ ]:
all_sequences, labels, exercise_angles = load_dataset(DATASET_PATH)

# Preprocessing

In [ ]:
processed_sequences, numerical_labels, label_angles = preprocess_data(all_sequences, labels, exercise_angles)

# Tensorflow Dataset and Model Routines

# K-Fold Cross-Validation

In [ ]:
def split_by_fold(data_x, data_y, folds, test):
    train_x, test_x, train_y, test_y = [], [], [], []
    for fold, split in enumerate(folds):
        for sample in split:
            if fold == test:
                test_x.append(data_x[sample])
                test_y.append(data_y[sample])
            else:
                train_x.append(data_x[sample])
                train_y.append(data_y[sample])
    return train_x, test_x, train_y, test_y

In [ ]:
# Preparing the 5 folds
k = 5
N = len(numerical_labels)
indexes = np.arange(N)
np.random.seed(42)
np.random.shuffle(indexes)

fold_size = int(np.ceil(N / k))

folds = []
for i in range(k):
    start = i * fold_size
    end = min(start + fold_size, N)
    folds.append(indexes[start:end])

print(folds)

In [ ]:
for seq_module in ['gru', 'lstm', 'transformer']:
    print(f'Performing k-fold cross-validation for the Angle Usefulness {seq_module} ...')
    for test in range(k):
        print(f'Testing on Fold #{test+1}/{k} ...')
        X_train, X_test, y_train, y_test = split_by_fold(
            processed_sequences,
            numerical_labels,
            folds,
            test
        )

        X_train, X_val, y_train, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.2,
            random_state=42
        )

        y_train = np.array(y_train, dtype=np.int32)
        y_test = np.array(y_test, dtype=np.int32)
        y_val = np.array(y_val, dtype=np.int32)

        Y_train = np.empty((len(X_train), len(X_train[0][0])), np.float32)
        Y_val = np.empty((len(X_val), len(X_train[0][0])), np.float32)
        Y_test = np.empty((len(X_test), len(X_train[0][0])), np.float32)

        for (Y, y) in [(Y_train, y_train), (Y_val, y_val), (Y_test, y_test)]:
            for i in range(Y.shape[0]):
                for angle in range(Y.shape[1]):
                    if angle in label_angles[y[i]]:
                        Y[i, angle] = 1
                    else:
                        Y[i, angle] = 0

        usefulness_model = angle_usefulness_model(maxlen=MAXLEN, module=seq_module, lr=1e-3)

        pbar_callback = TQDMProgressBar(epochs=80)

        callback = tf.keras.callbacks.ModelCheckpoint(
            MODEL_SAVE_PATH_USEFULNESS,
            monitor="val_binary_accuracy",
            verbose=0,
            save_best_only=True,
            save_weights_only=False,
            mode="max",
            save_freq="epoch",
            initial_value_threshold=None,
        )

        train_ds = UsefulnessDataset(X_train, Y_train, y_train, batch_size=batch_size, maxlen=MAXLEN, train=True)
        val_ds = UsefulnessDataset(X_val, Y_val, y_val, batch_size=min(batch_size, len(X_val)), maxlen=MAXLEN, train=False)

        history = usefulness_model.fit(
            train_ds,
            validation_data=val_ds,
            batch_size=batch_size,
            epochs=80,
            callbacks=[callback, pbar_callback],
            verbose=0, # Change this to 1 if you want progressive display while training
        )

        # Load the trained classifier model
        usefulness_model = tf.keras.models.load_model(
            MODEL_SAVE_PATH_USEFULNESS
        )

        test_ds = UsefulnessDataset(X_test, Y_test, y_test, batch_size=batch_size, maxlen=MAXLEN, train=False)
        usefulness_model.evaluate(test_ds, verbose=1)

In [ ]:
for seq_module in ['gru', 'lstm', 'transformer']:
    print(f'Performing k-fold cross-validation for the Siamese {seq_module} ...')
    for test in range(k):
        print(f'Testing on Fold #{test+1}/{k} ...')
        X_train, X_test, y_train, y_test = split_by_fold(
            processed_sequences,
            numerical_labels,
            folds,
            test
        )

        X_train, X_val, y_train, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.2,
            random_state=42
        )

        y_train = np.array(y_train, dtype=np.int32)
        y_test = np.array(y_test, dtype=np.int32)
        y_val = np.array(y_val, dtype=np.int32)

        train_batches = 10 * len(X_train) // batch_size
        val_batches = 10 * len(X_val) // batch_size
        train_ds = SimilarityDataset(X_train, y_train, num_batches=train_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=True)
        val_ds = SimilarityDataset(X_val, y_val, num_batches=val_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=False)

        callback = tf.keras.callbacks.ModelCheckpoint(
            MODEL_SAVE_PATH_SIMILARITY,
            monitor="val_binary_accuracy",
            verbose=0,
            save_best_only=True,
            save_weights_only=False,
            mode="max",
            save_freq="epoch",
            initial_value_threshold=None,
        )

        pbar_callback = TQDMProgressBar(epochs=60)

        # Build and train Siamese model
        similarity_model = angle_similarity_model(maxlen=MAXLEN, module=seq_module, lr=1e-3)
        history = similarity_model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=60,
            callbacks=[callback, pbar_callback],
            verbose=0,  # Change this to 1 if you want progressive display while training
        )

        similarity_model = tf.keras.models.load_model(
            MODEL_SAVE_PATH_SIMILARITY,
        )

        # Make a large dataset in terms of pair in order to have a more reliable test
        # since we can't test on all possible pairs (very huge number)
        test_batches = 1000 * len(X_test) // batch_size
        # Evaluate the Siamese model on the test set
        test_ds = SimilarityDataset(X_test, y_test, num_batches=test_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=False)
        similarity_model.evaluate(test_ds, verbose=1)

# One-shot Generalization Evaluation

In [ ]:
tests = [[3, 4, 13, 14, 15, 17], [1, 3, 4, 13, 14, 20], [1, 3, 6, 13, 19, 20], [1, 3, 7, 13, 14, 16], [1, 3, 5, 13, 14, 19], [1, 2, 3, 13, 19, 21], [0, 3, 6, 7, 13, 19], [1, 3, 4, 13, 14, 21], [3, 7, 13, 14, 16, 17], [1, 3, 10, 13, 20, 21]]

In [ ]:
# Function to split data by specifying test labels
def split_by_labels(data_x, data_y, test_labels):
    X_train, X_test, y_train, y_test = [], [], [], []
    for sample_x, sample_y in zip(data_x, data_y):
        if sample_y in test_labels:
            X_test.append(sample_x)
            y_test.append(sample_y)
        else:
            X_train.append(sample_x)
            y_train.append(sample_y)
    return X_train, X_test, y_train, y_test

In [ ]:
print('Performing one-shot validation for the Angle Usefulness Transformer ...')
for test in tests:
    print(f'Testing on Exercises: {test} ...')
    X_train, X_test, y_train, y_test = split_by_labels(
        processed_sequences,
        numerical_labels,
        test_labels=test,
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_train,
        y_train,
        test_size=0.2,
        random_state=42
    )

    y_train = np.array(y_train, dtype=np.int32)
    y_test = np.array(y_test, dtype=np.int32)
    y_val = np.array(y_val, dtype=np.int32)

    Y_train = np.empty((len(X_train), len(X_train[0][0])), np.float32)
    Y_val = np.empty((len(X_val), len(X_train[0][0])), np.float32)
    Y_test = np.empty((len(X_test), len(X_train[0][0])), np.float32)

    for (Y, y) in [(Y_train, y_train), (Y_val, y_val), (Y_test, y_test)]:
        for i in range(Y.shape[0]):
            for angle in range(Y.shape[1]):
                if angle in label_angles[y[i]]:
                    Y[i, angle] = 1
                else:
                    Y[i, angle] = 0

    usefulness_model = angle_usefulness_model(maxlen=MAXLEN, module='transformer', lr=1e-4)

    callback = tf.keras.callbacks.ModelCheckpoint(
        MODEL_SAVE_PATH_USEFULNESS,
        monitor="val_binary_accuracy",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        mode="max",
        save_freq="epoch",
        initial_value_threshold=None,
    )

    pbar_callback = TQDMProgressBar(epochs=80)

    train_ds = UsefulnessDataset(X_train, Y_train, y_train, batch_size=batch_size, maxlen=MAXLEN, train=True)
    val_ds = UsefulnessDataset(X_val, Y_val, y_val, batch_size=min(batch_size, len(X_val)), maxlen=MAXLEN, train=False)

    history = usefulness_model.fit(
        train_ds,
        validation_data=val_ds,
        batch_size=batch_size,
        epochs=80,
        callbacks=[callback, pbar_callback],
        verbose=0,  # Change this to 1 if you want progressive display while training
    )

    # Load the trained classifier model
    usefulness_model = tf.keras.models.load_model(
        MODEL_SAVE_PATH_USEFULNESS
    )

    test_ds = UsefulnessDataset(X_test, Y_test, y_test, batch_size=batch_size, maxlen=MAXLEN, train=False)
    usefulness_model.evaluate(test_ds, verbose=1)

In [ ]:
print('Performing one-shot validation for the Siamese Transformer ...')
for test in tests:
    print(f'Testing on Exercises: {test} ...')

    X_train, X_test, y_train, y_test = split_by_labels(
        processed_sequences,
        numerical_labels,
        test_labels=test,
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_train,
        y_train,
        test_size=0.2,
        random_state=42
    )
    train_batches = 10 * len(X_train) // batch_size
    val_batches = 10 * len(X_val) // batch_size
    train_ds = SimilarityDataset(X_train, y_train, num_batches=train_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=True)
    val_ds = SimilarityDataset(X_val, y_val, num_batches=val_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=False)

    callback = tf.keras.callbacks.ModelCheckpoint(
        MODEL_SAVE_PATH_SIMILARITY,
        monitor="val_binary_accuracy",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        mode="max",
        save_freq="epoch",
        initial_value_threshold=None,
    )

    pbar_callback = TQDMProgressBar(epochs=60)

    # Build and train Siamese model
    similarity_model = angle_similarity_model(maxlen=MAXLEN, module='transformer', lr=1e-3)
    history = similarity_model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=60,
        callbacks=[callback, pbar_callback],
        verbose=0,  # Change this to 1 if you want progressive display while training
    )

    # Load the trained Siamese model
    similarity_model = tf.keras.models.load_model(
        MODEL_SAVE_PATH_SIMILARITY,
    )

    # Make a large dataset in terms of pair in order to have a more reliable test
    # since we can't test on all possible pairs (very huge number)
    test_batches = 1000 * len(X_test) // batch_size
    # Evaluate the Siamese model on the test set
    test_ds = SimilarityDataset(X_test, y_test, num_batches=test_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=False)
    similarity_model.evaluate(test_ds, verbose=1)